# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
# Import essential packages
import os
# the glob module is used to retrieve files/pathnames 
# matching a specified pattern. The pattern rules of glob 
# follow standard Unix path expansion rules. It is also predicted 
# that according to benchmarks it is faster than other methods to 
# match pathnames in directories
import glob
import psycopg2 as postgres
import pandas as pd

# Import all queries from sql_queries.py
from sql_queries import *

In [2]:
# Set connection string variables
hostname = "127.0.0.1"
database_name = "sparkifydb"
username = "postgres"
password = "123456"
    
# Create a connection to Sparkify database
connection = postgres.connect(
    "host={} dbname={} user={} password={}".format(
        hostname, database_name, username, password
    )
)

# Create a cursor to execute queries on Sparkify DB
cursor = connection.cursor()

In [3]:
def get_files(filepath):
    """Get the absolute paths of all files inside a specific path

    Args:
        filepath (str): The path which contains all files we need

    Returns:
        [list]: list of all absolute paths
    """
    
    # Create empty list to hold all filenames with their absolute path
    # that exist in a specific directory
    all_files = []
    
    # root : directories only from the path we specified.
    # dirs : sub-directories from root.
    # files : Prints out all files from root and directories.
    for root, dirs, files in os.walk(filepath):
        
        # We can use the function glob.glob() or glob.iglob() directly 
        # from glob module to retrieve paths recursively from inside the 
        # directories/files and subdirectories/subfiles.
        files = glob.glob(os.path.join(root,'*.json'))
        
        # loop over the files
        for f in files :
            
            # Append the absolute path of each file to the all_files list
            all_files.append(os.path.abspath(f))
    
    # Return the list of all absolute paths
    return all_files

# Process `song_data`
In this first part, we'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
# Get the list of all absolute paths inside the dataset directory
files_list = get_files('data/song_data/')

In [5]:
# Get the absolute path of the first file; as a proof of concept
file_absolute_path = files_list[0]
print(file_absolute_path)

E:\Repositories\public\Sparkify-ETL-pipeline-project\data\song_data\A\A\A\TRAAAAW128F429D538.json


In [6]:
# Create a dataframe of the Proof of Conecpt file
df_songs_data = pd.read_json(file_absolute_path, lines=True)
df_songs_data.head(5)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
# Select the columns we need
songs_table_df = df_songs_data[['song_id', 'title', 'artist_id', 'year', 'duration']]
songs_table_df

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179


In [8]:
# Get values of the first record
first_song_array = songs_table_df.values[0] # Returns an array

# Convert the array to a list
song_data = first_song_array.tolist()
print(song_data, type(song_data))

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179] <class 'list'>


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
# insert the record into the song table
cursor.execute(songs_table_insert, song_data)
connection.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
# Select the columns we need for the artists table
artists_df = df_songs_data[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artists_df

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN


In [11]:
# Get the first row into a list to insert it in the artists table
artist_data = artists_df.values[0].tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

In [27]:
# Loop over the rows in 'time_df' dataframe
for i, row in artists_df.iterrows():
    print(row)

artist_id           ARD7TVE1187B99BFB1
artist_name                     Casual
artist_location        California - LA
artist_latitude                    NaN
artist_longitude                   NaN
Name: 0, dtype: object


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
cursor.execute(artists_table_insert, artist_data)
connection.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
# Get the absolute paths of all files in the log_data directory
log_files_list = get_files('data/log_data/')

In [14]:
# Get first file absolute path from the list
file_absolute_path = log_files_list[0]
file_absolute_path

'E:\\Repositories\\public\\Sparkify-ETL-pipeline-project\\data\\log_data\\2018\\11\\2018-11-01-events.json'

In [15]:
# Convert the file to a dataframe
df_log_data = pd.read_json(file_absolute_path, lines=True)
df_log_data.head(5)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
# We filter the dataframe records to extract the records with a value of 'NextSong' in the 'page' column
df_log_data = df_log_data[df_log_data.page == 'NextSong'].reset_index(drop = True)
df_log_data.head(5)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
1,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [17]:
# Convert the 'ts' column to datetime format
df_log_data['start_time'] = pd.to_datetime(df_log_data.ts, unit = 'ms')

# Now we can extract the hour, day, week of year, month, year, and weekday from 'start_time' column 
df_log_data['year'] = df_log_data['start_time'].dt.year
df_log_data['month'] = df_log_data['start_time'].dt.month
df_log_data['week'] = df_log_data['start_time'].dt.isocalendar().week
df_log_data['weekday'] = df_log_data['start_time'].dt.dayofweek
df_log_data['day'] = df_log_data['start_time'].dt.day
df_log_data['hour'] = df_log_data['start_time'].dt.hour

# Create `time_df` dataframe containing the time data for this file
time_df = df_log_data[['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']]
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-01 21:01:46.796,21,1,44,11,2018,3
1,2018-11-01 21:05:52.796,21,1,44,11,2018,3
2,2018-11-01 21:08:16.796,21,1,44,11,2018,3
3,2018-11-01 21:11:13.796,21,1,44,11,2018,3
4,2018-11-01 21:17:33.796,21,1,44,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
# Loop over the rows in 'time_df' dataframe
for i, row in time_df.iterrows():
    
    # Insert the current row in 'time' table
    cursor.execute(time_table_insert, list(row))
    # Commit the action
    connection.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
# Print all columns names to extract the columns names that we want
df_log_data.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId',
       'start_time', 'year', 'month', 'week', 'weekday', 'day', 'hour'],
      dtype='object')

In [20]:
# Extract the columns we want into a new dataframe 'user_df'
user_df = df_log_data[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head(5)

,userId,firstName,lastName,gender,level
0,8,Kaylee,Summers,F,free
1,8,Kaylee,Summers,F,free
2,8,Kaylee,Summers,F,free
3,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free


In [21]:
# We notice that there are many duplicates
# However, We only want one record for each user

# So, we drop the duplicated rows
user_df = user_df.drop_duplicates().sort_values(by=['userId']).reset_index(drop = True)
user_df.head()

,userId,firstName,lastName,gender,level
0,8,Kaylee,Summers,F,free
1,10,Sylvie,Cruz,F,free
2,26,Ryan,Smith,M,free
3,101,Jayden,Fox,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [22]:
# Loop over the rows in 'user_df' and insert each record into 'users' table
for i, row in user_df.iterrows():
    
    # Execute the INSERT query for the current record
    cursor.execute(users_table_insert, row)
    # Commit the action
    connection.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [23]:
# Loop over the records in the log file dataframe
for index, row in df_log_data.iterrows():

    # get song_id and artist_id from song and artist tables
    
    # The query looks like this:
    #################
    # SELECT s.song_id, a.artist_id
    # FROM songs s
    # JOIN artists a
    # ON s.artist_id = a.artist_id
    # WHERE s.title = %s AND a.name = %s AND s.duration = %s
    #################
    # As you see, we already put placeholders in the query;
    # to run the WHERE statements with dynamic values
    # These values, we get it from each record in this loop
    
    # So, we execute the SELECT query with the variables of the
    # current record
    cursor.execute(song_select, (row.song, row.artist, row.length))
    
    # Since we expect one result, we fetch one row from the results
    results = cursor.fetchone()
    
    # We check if there are any results
    if results:
        
        # If so, we unpack the results into two variables;
        # since we only selected two variables in the query
        song_id, artist_id = results
        
    else:
        # If there are no results, we set the two variables to none
        song_id, artist_id = None, None

    # insert songplay record
    songplay_data = (row.start_time, row.userId, row.level, song_id, artist_id, row.sessionId, row.location, row.userAgent)
    
    # Execute the query and insert the data into 'songplays' table
    cursor.execute(songplay_table_insert, songplay_data)
    # Commit the action
    connection.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close connectionion to Sparkify Database

In [24]:
# Close cursor
cursor.close()

In [25]:
# Close the connection
connection.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

### os.walk() in Python
How to traverse file system in Python ? Suppose we have given below file structure in our system and we want to traverse all it’s branches completely from top to bottom ?

![File Structure](images/osWalk.jpg)

### How does os.walk() work in python ?

OS.walk() generate the file names in a directory tree by walking the tree either top-down or bottom-up. For each directory in the tree rooted at directory top (including top itself), it yields a 3-tuple (dirpath, dirnames, filenames).

* root : Prints out directories only from what you specified.
* dirs : Prints out sub-directories from root.
* files : Prints out all files from root and directories.

**Reference:**  
https://www.geeksforgeeks.org/os-walk-python/